In [2]:
import os
import numpy as np

In [16]:
import os
import torch
import glob
import numpy as np 
import random
import math
from os import listdir
from os.path import isfile, join
from torch.utils.data import Dataset as Dataset_n
from torch_geometric.data import DataLoader as DataLoader_n
from torch_geometric.data import Data

def bump(g):
    return g
    # return Data.from_dict(g.__dict__)

class LabelledDataset(Dataset_n):
    def __init__(self, npy_file, processed_dir):
      self.npy_ar = np.load(npy_file)
      self.processed_dir = processed_dir
      self.protein_1 = self.npy_ar[:,2]
      self.protein_2 = self.npy_ar[:,5]
      self.label = self.npy_ar[:,6].astype(float)
      self.n_samples = self.npy_ar.shape[0]

    def __len__(self):
      return(self.n_samples)

    def __getitem__(self, index):
      prot_1 = os.path.join(self.processed_dir, self.protein_1[index]+".pt")
      prot_2 = os.path.join(self.processed_dir, self.protein_2[index]+".pt")
      prot_1 = torch.load(glob.glob(prot_1)[0])['mean_representations'][33]
      prot_2 = torch.load(glob.glob(prot_2)[0])['mean_representations'][33]
      prot_1 = bump(prot_1)
      prot_2 = bump(prot_2)
      return prot_1, prot_2, torch.tensor(self.label[index])

In [29]:
npy_file = "../human_features/npy_file_new(human_dataset).npy"
npy_file_test = "../human_features/npy_file_new(human_dataset)_test.npy"
processed_dir="../human_features/esm/"
# train = LabelledDataset(npy_file=npy_file, processed_dir=processed_dir)
# test = LabelledDataset(npy_file=npy_file_test, processed_dir=processed_dir)

In [45]:
from tqdm.auto import tqdm

X_train = []
y_train = []
mapping_train = np.load(npy_file)
bad_rows = []
for row in tqdm(mapping_train):
    try:
        _, _, prot1, _, _, prot2, label = row
        prot_1 = os.path.join(processed_dir, prot1+".pt")
        prot_2 = os.path.join(processed_dir, prot2+".pt")
        prot_1 = torch.load(glob.glob(prot_1)[0])['mean_representations'][33]
        prot_2 = torch.load(glob.glob(prot_2)[0])['mean_representations'][33]
        p = torch.cat([prot_1, prot_2])
        label = float(label)
        X_train.append(p)
        y_train.append(label)
    except:
        bad_rows.append(row)

  0%|          | 0/98396 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [70]:
X_test = []
y_test = []
bad_test_rows = []
mapping_test = np.load(npy_file_test)
for row in tqdm(mapping_test):
    try:
        _, _, prot1, _, _, prot2, label = row
        prot_1 = os.path.join(processed_dir, prot1+".pt")
        prot_2 = os.path.join(processed_dir, prot2+".pt")
        prot_1 = torch.load(glob.glob(prot_1)[0])['mean_representations'][33]
        prot_2 = torch.load(glob.glob(prot_2)[0])['mean_representations'][33]
        p = torch.cat([prot_1, prot_2])
        label = float(label)
        X_test.append(p)
        y_test.append(label)
    except:
        bad_test_rows.append(row)

  0%|          | 0/5677 [00:00<?, ?it/s]

In [49]:
for i in range(len(X_train)):
    X_train[i] = X_train[i][None, :]

In [52]:
X_train = torch.cat(X_train)

In [59]:
from sklearn.model_selection import train_test_split

train, val, ytrain, yval = train_test_split(X_train, y_train, test_size=0.2)

In [60]:
ytrain = [x > 0 for x in ytrain]
yval = [x > 0 for x in yval]

In [63]:
from sklearn.linear_model import LogisticRegression

In [64]:
logreg = LogisticRegression().fit(train, ytrain)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [67]:
from sklearn.metrics import accuracy_score
accuracy_score(yval, logreg.predict(val))

0.9602628763564114

In [72]:
for i in range(len(X_test)):
    X_test[i] = X_test[i][None, :]
X_test = torch.cat(X_test)
y_test = [x > 0 for x in y_test]

In [78]:
X_test = X_test.squeeze(1)

In [79]:
accuracy_score(y_test, logreg.predict(X_test))

0.9393564356435643

In [80]:
from collections import Counter
Counter(y_test)

Counter({True: 5203, False: 453})

In [82]:
5203 / (453 + 5203) 

0.9199080622347949